In [1]:
import os
import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from data import WhaleData
from model import Net
from loss import FocalLossQb, bce_loss
from eval import do_valid

In [2]:
batch_size = 32

out_dir = os.path.join('./models/', 'resnet101')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
if not os.path.exists(os.path.join(out_dir,'checkpoint')):
    os.makedirs(os.path.join(out_dir,'checkpoint'))
if not os.path.exists(os.path.join(out_dir,'train')):
    os.makedirs(os.path.join(out_dir,'train'))

        
train_dataset = WhaleData(mode='train')
valid_0 = WhaleData(mode='valid', fold_id=0)
valid_1 = WhaleData(mode='valid', fold_id=1)
valid_2 = WhaleData(mode='valid', fold_id=2)
valid_3 = WhaleData(mode='valid', fold_id=3)
valid_4 = WhaleData(mode='valid', fold_id=4)
valid_5 = WhaleData(mode='valid', fold_id=5)

In [3]:
valid_loader0  = DataLoader(valid_0, batch_size=batch_size, drop_last=False, num_workers=16)
valid_loader1  = DataLoader(valid_1, batch_size=batch_size, drop_last=False, num_workers=16)
valid_loader2  = DataLoader(valid_2, batch_size=batch_size, drop_last=False, num_workers=16)
valid_loader3  = DataLoader(valid_3, batch_size=batch_size, drop_last=False, num_workers=16)
valid_loader4  = DataLoader(valid_4, batch_size=batch_size, drop_last=False, num_workers=16)
valid_loader5  = DataLoader(valid_4, batch_size=batch_size, drop_last=False, num_workers=16)

In [4]:
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")

net = Net(num_class=5005)
for p in net.basemodel.layer0.parameters(): 
    p.requires_grad = False
    
for p in net.basemodel.layer1.parameters(): 
    p.requires_grad = False
    
for p in net.basemodel.layer2.parameters(): 
    p.requires_grad = False
    
net = torch.nn.DataParallel(net)
net = net.to(device)
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()),
                             lr=0.0001, betas=(0.9, 0.999),
                             eps=1e-08, weight_decay=0.0002)

In [ ]:
EPOCH=10
i=0
batch_size=32
device = 'cuda'

for epoch in range(EPOCH):
    train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, drop_last=False, num_workers=16)
    for input, truth_ in train_loader:
        i+=1
        input = input.to(device)
        truth_ = truth_.to(device)
        optimizer.zero_grad()
        logit = net(input)
        batch_loss1 = FocalLossQb(gamma=2)(logit, truth_)
        batch_loss2 = bce_loss(logit, truth_)
        loss = batch_loss1+batch_loss2
        loss.backward()
        optimizer.step()
        
        if i % 10 == 0:
            net.eval()
            train_loss1, train_loss2, train_loss, train_acc, label_5, pred_5 = do_valid(net, train_loader, device=device) 
            valid_loss1, valid_loss2, valid_loss, valid_acc, label_5_val, pred_5_val = do_valid(net, valid_loader0, device=device)
            net.train()
            print('--------------------------------------------------------------------')
            print('train_loss1:{:.4f} || train_loss2:{:.4f} || train_loss:{:.4f} || train_acc:{:.4f} ||'.format(train_loss1, train_loss2, train_loss, train_acc))
            print('valid_loss1:{:.4f} || valid_loss2:{:.4f} || valid_loss:{:.4f} || valid_acc:{:.4f} ||'.format(valid_loss1, valid_loss2, valid_loss, valid_acc))
            print('five_sample_label:\n{}\nfive_sample_predict:\n{}\n'.format(label_5, pred_5))
            